# Pruning defense

Prune an EfficientNetv2 model that has been pretrained on the CIFAR-10 dataset.

In [ ]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchvision.utils import make_grid

In [ ]:
from Data import Data

In [ ]:
DATASETS_DIR = os.path.join("..", "datasets")
CIFAR_DIR = os.path.join(DATASETS_DIR, "CIFAR10", "cifar-10")

MODEL_NAME = 'efficientnet_v2_s'
WEIGHT_PATH = os.path.join("..", "models", "efficientnet_v2_s_cifar10.pth")

## Load locally stored CIFAR-10

We'll just be using the clean dataset for now.

In [ ]:
train_images = os.path.join(CIFAR_DIR, "train", "data.npy")
train_labels = os.path.join(CIFAR_DIR, "train", "labels.npy")
test_images = os.path.join(CIFAR_DIR, "test", "data.npy")
test_labels = os.path.join(CIFAR_DIR, "test", "labels.npy")

cifar_10_dataset= Data(train_images=train_images,train_labels=train_labels,
                     test_images=test_images,test_labels=test_labels)

In [ ]:
# cifar_10_dataset.normalize()
cifar_10_dataset.show_images()

## Load the pretrained weights

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = torch.hub.load('hankyul2/EfficientNetV2-pytorch', MODEL_NAME, nclass=cifar_10_dataset.num_classes, skip_validation=False)
model.to(device)

state_dict = torch.load(WEIGHT_PATH)
model.load_state_dict(state_dict)

device

## Visualize activations of ~~the last convolutional layer~~ any layer

> "Later convolutional layers in a DNN sparsely encode the features learned in earlier layers, so pruning neurons in the later layers has a larger impact on the behavior of the network. Consequently, **we prune only the last convolutional layer**..."


In [ ]:
# Available layers
list(model.state_dict().keys())

In [ ]:
SELECTED_WEIGHT_KEY = 'blocks.0.block.fused.0.weight'

In [ ]:
layer_weights = model.state_dict()[SELECTED_WEIGHT_KEY]

layer_weights.size()

In [ ]:
# Only visualize the first out of 24 channels

grid = make_grid(layer_weights[:,:1,:,:], pad_value=1)
np_grid = grid.cpu().numpy()
plt.imshow(np.transpose(np_grid, (1, 2, 0)))
plt.show()

# ...

In [ ]:
print(model)

# Test model on test dataset (measure accuracy) for comparison after pruning 

In [ ]:
from torch.utils.data import Dataset, DataLoader, random_split,TensorDataset
from torchvision import transforms 
from torchvision.transforms import *
from tqdm import tqdm

In [ ]:
cifar_10_dataset.normalize()

In [ ]:
test_data = torch.tensor(cifar_10_dataset.test_images, dtype=torch.float32).permute(0, 3, 1, 2)
test_labels = torch.tensor(cifar_10_dataset.test_labels, dtype=torch.long)
test_dataset = TensorDataset(test_data, test_labels)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
def evaluate_model(model, data_loader, device):
    model.eval()  
    total = 0
    correct = 0
    i = 0
    resize_transform = transforms.Resize((224, 224), antialias=True)

    with torch.no_grad():
        for images, labels in tqdm(data_loader):
            # Resize images here if facing memory issues with whole dataset reshaped at once
            images = torch.stack([resize_transform(img) for img in images])
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            i = i + 1

    accuracy = 100 * correct / total
    return accuracy

In [ ]:
accuracy = evaluate_model(model, test_loader, device)
print(f'Test Accuracy: {accuracy}%')


## Prune last convolutional layer in Efficient Net and evaluate new model

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
prune_rate = 0.2

In [ ]:
train_data = torch.tensor(cifar_10_dataset.train_images, dtype=torch.float32).permute(0, 3, 1, 2)
train_labels = torch.tensor(cifar_10_dataset.train_labels, dtype=torch.long)
train_dataset = TensorDataset(train_data, train_labels)


In [ ]:
tr_loader = DataLoader(train_dataset, batch_size=64,
                               num_workers=8 ,
                               drop_last=True, pin_memory=True)

In [ ]:
layer_to_prune = None
for name, module in model.named_modules():
    if name == 'blocks.39.block.depth_wise.0':
        layer_to_prune = module
        break

if layer_to_prune is None:
    raise AttributeError("Layer 'blocks.39.block.depth_wise.0' not found in the model")


In [ ]:
print("Layer to prune:", layer_to_prune)
print("======== pruning... ========")
with torch.no_grad():
   container = []
   def forward_hook(module, input, output):
      container.append(output)
   #hook = getattr(model, layer_to_prune).register_forward_hook(forward_hook)
   hook = layer_to_prune.register_forward_hook(forward_hook)
   print("Forwarding all training set")
   model.eval()
   for data, _ in tr_loader:
      if device =="cuda":
         model(data.cuda())
      else:
         model(data)
   hook.remove()

In [ ]:
print(container)

In [ ]:
container_copy = container
print(container==container_copy)

In [ ]:
squeezed_outputs = [torch.squeeze(output) for output in container]
container_squeezed =  torch.cat(squeezed_outputs, dim=0)


In [ ]:
print(container_squeezed)

In [ ]:
container = torch.cat(container, dim=0)

In [ ]:
activation = torch.mean(container, dim=[0, 2, 3])
# activation = torch.mean(container_squeezed)
print(activation)

In [ ]:
seq_sort = torch.argsort(activation)
print(seq_sort)

In [ ]:
num_channels = len(activation)
print(num_channels)

In [ ]:
prunned_channels = int(num_channels * prune_rate)
mask = torch.ones(num_channels).to(device)

In [ ]:
for element in seq_sort[:prunned_channels]:
   mask[element] = 0
if len(container.shape) == 4:
   mask = mask.reshape(1, -1, 1, 1)
   
if len(layer_to_prune.weight.data.shape) == 4:
    mask = mask.view(1, -1, 1, 1)

In [ ]:
layer_to_prune.weight.data *= mask
model = model
print("======== pruning complete ========")

In [ ]:
accuracy = evaluate_model(model, test_loader, device)
print(f'Test Accuracy: {accuracy}%')
